### Load data

In [1]:
import pandas as pd

df = pd.read_csv(r'df_almost_ready.csv')
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V22,V23,V24,V25,V26,V27,V28,Amount,Class,Time_hours
0,-1.071719,-4.343904,-0.497727,-0.277591,-2.482346,-0.472073,0.991747,-0.414207,1.694252,-1.744176,...,0.013647,-1.183439,0.536058,0.064148,-0.016945,-0.179133,0.236036,35.112713,0,10.356389
1,1.314915,-0.980378,-0.032665,-2.770975,-1.047365,-0.705180,-0.491240,-0.076395,0.571959,-0.673573,...,-0.481886,-0.009117,0.009622,0.555844,-0.836622,0.099256,0.014826,26.310000,0,13.365278
2,-1.680526,-0.959150,-1.329705,-1.028361,-0.730922,0.253124,3.308312,-0.480062,0.157630,-2.012017,...,-0.204108,0.517400,-0.611013,-0.445912,-0.409744,0.231809,0.253554,35.112713,0,35.431667
3,2.114488,0.169976,-2.051966,0.070326,0.481642,-1.819440,0.886700,-0.673729,0.077960,-0.006214,...,0.642597,-0.058720,0.128632,0.461792,0.234073,-0.082360,-0.075323,12.920000,0,42.215278
4,-5.277692,3.487837,-1.201844,-0.192904,-0.158582,0.949549,0.467901,-0.788434,4.652615,7.407039,...,-1.077669,0.160074,-1.535126,0.873245,-0.322503,1.383375,0.758996,28.900000,0,18.243056


### Spliting the data into x and y

In [2]:
x = df.drop(columns='Class', axis=1 )

x.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Time_hours
0,-1.071719,-4.343904,-0.497727,-0.277591,-2.482346,-0.472073,0.991747,-0.414207,1.694252,-1.744176,...,0.856441,0.013647,-1.183439,0.536058,0.064148,-0.016945,-0.179133,0.236036,35.112713,10.356389
1,1.314915,-0.980378,-0.032665,-2.770975,-1.047365,-0.705180,-0.491240,-0.076395,0.571959,-0.673573,...,-0.416346,-0.481886,-0.009117,0.009622,0.555844,-0.836622,0.099256,0.014826,26.310000,13.365278
2,-1.680526,-0.959150,-1.329705,-1.028361,-0.730922,0.253124,3.308312,-0.480062,0.157630,-2.012017,...,-0.084593,-0.204108,0.517400,-0.611013,-0.445912,-0.409744,0.231809,0.253554,35.112713,35.431667
3,2.114488,0.169976,-2.051966,0.070326,0.481642,-1.819440,0.886700,-0.673729,0.077960,-0.006214,...,0.163748,0.642597,-0.058720,0.128632,0.461792,0.234073,-0.082360,-0.075323,12.920000,42.215278
4,-5.277692,3.487837,-1.201844,-0.192904,-0.158582,0.949549,0.467901,-0.788434,4.652615,7.407039,...,-0.477313,-1.077669,0.160074,-1.535126,0.873245,-0.322503,1.383375,0.758996,28.900000,18.243056


In [3]:

y = df['Class']


### Splittinng into train set and test set

In [4]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)


## Logistic Regression Model

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report
import warnings
# Filter out future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [13]:
# Creating a pipeline that scales the data and then applies Logistic Regression
pipeline = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000))


In [29]:
# Define the parameter grid for grid search
param_grid = {
    'logisticregression__C': [0.001, 0.01, 0.1, 1, 10, 30, 50, 80, 100],
    'logisticregression__penalty': ['l2'],
    'logisticregression__class_weight': [ None,'balanced'] 
}

In [30]:
# Create GridSearchCV object
grid_search = GridSearchCV(pipeline, param_grid, cv=10, scoring='roc_auc')

# Fit the grid search to the data
grid_search.fit(x_train, y_train)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('logisticregression',
                                        LogisticRegression(max_iter=1000))]),
             param_grid={'logisticregression__C': [0.001, 0.01, 0.1, 1, 10, 30,
                                                   50, 80, 100],
                         'logisticregression__class_weight': [None, 'balanced'],
                         'logisticregression__penalty': ['l2']},
             scoring='roc_auc')

In [31]:
# Print the best parameters and the corresponding score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation ROC AUC:", grid_search.best_score_)


Best Parameters: {'logisticregression__C': 100, 'logisticregression__class_weight': None, 'logisticregression__penalty': 'l2'}
Best Cross-Validation ROC AUC: 0.9571964956195245


In [32]:
# Get the best model from grid search
best_model = grid_search.best_estimator_

# Perform cross-validation with the best estimator
scores = cross_val_score(best_model, x_train, y_train, cv=10, scoring='roc_auc') 
print("Cross-Validation ROC AUC Scores:", scores)
print("Average ROC AUC Score:", scores.mean())

Cross-Validation ROC AUC Scores: [1.         0.99249061 0.85732165 0.88110138 0.99499374 0.84605757
 1.         1.         1.         1.        ]
Average ROC AUC Score: 0.9571964956195245


    -Average ROC AUC Score: The average score from cross-validation is the same as the best cross-validation score because the cross-validation process is part of the grid search. This is a consistency check.

In [33]:

# Predict on the test data
y_pred = best_model.predict(x_test)

# Print the classification report for test metrics
print("Test Metrics Report:\n", classification_report(y_test, y_pred, target_names=['Not Fraud', 'Fraud']))


Test Metrics Report:
               precision    recall  f1-score   support

   Not Fraud       1.00      1.00      1.00      1997
       Fraud       0.25      0.33      0.29         3

    accuracy                           1.00      2000
   macro avg       0.62      0.67      0.64      2000
weighted avg       1.00      1.00      1.00      2000



### Insights

1. The model is likely overfitting to the majority class (Not Fraud), which is common in imbalanced datasets.
2. Precision for the 'Not Fraud' class is 1.00, meaning the model is excellent at ensuring that when it predicts a transaction is not fraud, it is correct practically every time while Precision for the 'Fraud' class is 0.25, this indicates that when the model predicts a transaction is fraudulent, it is correct only 25% of the time.
3. Recall for the 'Not Fraud' class is also 1.00, meaning the model identifies almost all non-fraudulent transactions correctly while Recall for the 'Fraud' class is 0.33, meaning the model correctly identifies 33% of all fraudulent transactions. This suggests that while it catches some frauds.
4. The F1-score for the 'Not Fraud' class is 1.00, indicating excellent precision and recall. The model's predictions for non-fraudulent transactions are reliable whle The F1-score for the 'Fraud' class is 0.29, which is quite low

In [34]:
lr_results = pd.DataFrame(['Logistic Regression', grid_search.best_score_]).transpose()
lr_results.columns = ['Method', 'Best c.v Accuracy']
lr_results

,Method,Best c.v Accuracy
0,Logistic Regression,0.957196


## Random Forest Model

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

In [36]:
# Initializing the RandomForestClassifier
randf_model = RandomForestClassifier(n_jobs=-1)

In [37]:

# Perform cross-validation on the training set with a more suitable scoring metric
scores = cross_val_score(randf_model, x_train, y_train, cv=10, scoring='roc_auc', n_jobs=-1)

In [38]:
# Print the scores for each fold and the average score
print("Cross-Validation ROC AUC Scores:", scores)
print("Average ROC AUC Score:", np.mean(scores))


Cross-Validation ROC AUC Scores: [1.         1.         0.48185232 0.48560701 0.99874844 0.48811014
 1.         1.         1.         1.        ]
Average ROC AUC Score: 0.8454317897371715


In [44]:
# Define the parameter grid for Random Forest
param_grid_rf = {
    'n_estimators': [10, 50, 100],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 4],
    'min_samples_leaf': [1, 2]
}

In [45]:
# Using RandomizedSearchCV
random_search_rf = RandomizedSearchCV(RandomForestClassifier(n_jobs=-1), 
                                      param_grid_rf, n_iter=10, cv=5, 
                                      scoring='roc_auc', n_jobs=-1, random_state=42)

random_search_rf.fit(x_train, y_train)

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(n_jobs=-1), n_jobs=-1,
                   param_distributions={'max_depth': [None, 10, 20],
                                        'min_samples_leaf': [1, 2],
                                        'min_samples_split': [2, 4],
                                        'n_estimators': [10, 50, 100]},
                   random_state=42, scoring='roc_auc')

In [46]:
# Print the best parameters and score
print("Best Parameters:", random_search_rf.best_params_)
print("Best ROC AUC:", random_search_rf.best_score_)

Best Parameters: {'n_estimators': 10, 'min_samples_split': 4, 'min_samples_leaf': 2, 'max_depth': 10}
Best ROC AUC: 0.8993634092814305


In [47]:
# Get the best model from grid search
best_rf_model = random_search_rf.best_estimator_

In [48]:

# Predict on the test data
y_pred = best_rf_model.predict(x_test)

In [49]:


# Print the classification report for test metrics
print("Test Metrics Report:\n", classification_report(y_test, y_pred, 
                                        target_names=['Not Fraud', 'Fraud']))


Test Metrics Report:
               precision    recall  f1-score   support

   Not Fraud       1.00      1.00      1.00      1997
       Fraud       0.75      1.00      0.86         3

    accuracy                           1.00      2000
   macro avg       0.88      1.00      0.93      2000
weighted avg       1.00      1.00      1.00      2000



### Insights

1. High Precision for 'Not Fraud': The model almost perfectly identifies 'Not Fraud' transactions, which is expected as they are the majority class.
2. High Precision and Recall for 'Fraud': The precision of 0.75 and recall of 1.00 for the 'Fraud' class are particularly noteworthy.
3. F1-Score for 'Fraud' (0.86): A high F1-score for the 'Fraud' class indicates a good balance between precision and recall.

In [50]:
rf_results = pd.DataFrame(['Random Forest', random_search_rf.best_score_]).transpose()
rf_results.columns = ['Method', 'Best c.v Accuracy']
rf_results

,Method,Best c.v Accuracy
0,Random Forest,0.899363


## Gradient Boosting Model

In [56]:
from sklearn.ensemble import GradientBoostingClassifier

In [57]:
# Initializing the GradientBoostingClassifier
gbc = GradientBoostingClassifier(n_estimators=50, learning_rate=1.5, max_depth=1, random_state=0)
gbc.fit(x_train, y_train)


GradientBoostingClassifier(learning_rate=1.5, max_depth=1, n_estimators=50,
                           random_state=0)

In [58]:
# Perform cross-validation on the training set
scores = cross_val_score(gbc, x_train, y_train, cv=5, scoring='roc_auc')  # Using ROC AUC as scoring metric

print("Cross-Validation Scores:", scores)
print("Average ROC AUC Score:", scores.mean())

Cross-Validation Scores: [0.99968711 0.5        0.66624922 1.         1.        ]
Average ROC AUC Score: 0.8331872652337025


In [67]:
# Define the parameter grid for grid search
param_grid_gb = {
    'n_estimators': [10,50, 90],
    'learning_rate': [0.1],
    'max_depth': [3, 5],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}


In [68]:
# Using RandomizedSearchCV
random_search_gb = RandomizedSearchCV(GradientBoostingClassifier(), param_grid_gb, 
                                      n_iter=10, cv=3, scoring='roc_auc', n_jobs=-1, random_state=42)
random_search_gb.fit(x_train, y_train)

RandomizedSearchCV(cv=3, estimator=GradientBoostingClassifier(), n_jobs=-1,
                   param_distributions={'learning_rate': [0.1],
                                        'max_depth': [3, 5],
                                        'min_samples_leaf': [1, 2],
                                        'min_samples_split': [2, 5],
                                        'n_estimators': [10, 50, 100]},
                   random_state=42, scoring='roc_auc')

In [69]:

print("Best Parameters:", random_search_gb.best_params_)
print("Best Cross-Validation ROC AUC:", random_search_gb.best_score_)


Best Parameters: {'n_estimators': 10, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 3, 'learning_rate': 0.1}
Best Cross-Validation ROC AUC: 0.8974705735036314


In [71]:

best_gb_model = random_search_gb.best_estimator_
y_pred_gb = random_search_gb.predict(x_test)


In [72]:

# Assuming you have the predicted probabilities for the positive class
y_pred_proba_gb = best_gb_model.predict_proba(x_test)[:, 1]
print("Test ROC AUC:", roc_auc_score(y_test, y_pred_proba_gb))

print("Test Metrics Report:\n", classification_report(y_test, y_pred_gb, target_names=['Not Fraud', 'Fraud']))

Test ROC AUC: 0.999248873309965
Test Metrics Report:
               precision    recall  f1-score   support

   Not Fraud       1.00      1.00      1.00      1997
       Fraud       0.50      0.67      0.57         3

    accuracy                           1.00      2000
   macro avg       0.75      0.83      0.79      2000
weighted avg       1.00      1.00      1.00      2000



### Insights

1. Precision for 'Fraud': At 0.50, the model's precision for the fraud class is moderate. This means that when it predicts a transaction as fraud, it is correct 50% of the time.
2. Recall for 'Fraud': The recall of 0.67 for the fraud class is more encouraging. It indicates the model is able to identify 67% of the actual fraud cases.
3. F1-Score for 'Fraud': The F1-score, which balances precision and recall, is 0.57 for the fraud class. This is a moderate score and suggests room for improvement, especially in terms of precision.
4. The average ROC AUC score from cross-validation is approximately 0.833, which is decent but not excellent.

In [73]:

gb_results = pd.DataFrame(['Gradient Boosting',  random_search_gb.best_score_]).transpose()
gb_results.columns = ['Method', 'Best c.v Accuracy']
gb_results


,Method,Best c.v Accuracy
0,Gradient Boosting,0.897471


## Making a neural network

In [13]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight


In [26]:
# Splitting the data
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)


In [27]:

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(enumerate(class_weights))



In [29]:
# Building the model
model = Sequential([
    Dense(256, activation='relu', input_shape=(x_train.shape[1],)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

In [30]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), 
              loss='binary_crossentropy', 
              metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])


In [31]:
# Train the model
history = model.fit(x_train, y_train, 
                    epochs=30, 
                    batch_size=32, 
                    validation_data=(x_val, y_val), 
                    class_weight=class_weights_dict, 
                    verbose=1)

Epoch 1/30
160/160 ━━━━━━━━━━━━━━━━━━━━ 26s 36ms/step - accuracy: 0.5533 - auc: 0.7231 - loss: 0.6268 - val_accuracy: 0.9211 - val_auc: 0.7594 - val_loss: 0.4066
Epoch 2/30
160/160 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.6677 - auc: 0.8612 - loss: 0.4546 - val_accuracy: 0.9969 - val_auc: 0.7866 - val_loss: 0.2497
Epoch 3/30
160/160 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.7216 - auc: 0.9734 - loss: 0.3317 - val_accuracy: 0.9969 - val_auc: 0.8658 - val_loss: 0.2012
Epoch 4/30
160/160 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.7600 - auc: 0.9558 - loss: 0.3654 - val_accuracy: 0.9977 - val_auc: 0.9757 - val_loss: 0.1855
Epoch 5/30
160/160 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.8021 - auc: 0.8914 - loss: 0.3507 - val_accuracy: 0.9984 - val_auc: 0.7502 - val_loss: 0.1142
Epoch 6/30
160/160 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.8222 - auc: 0.8437 - loss: 0.2500 - val_accuracy: 0.9977 - val_auc: 0.9832 - val_loss: 0.1156
Epoch 7/30
160/160 ━━━━━━━━━━━━━━

In [20]:
shallow_nn.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=5, callbacks=[checkpoint])


Epoch 1/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.5498 - loss: 0.6999 - val_accuracy: 0.8169 - val_loss: 0.4905
Epoch 2/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8559 - loss: 0.4698 - val_accuracy: 0.9950 - val_loss: 0.3228
Epoch 3/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9979 - loss: 0.2804 - val_accuracy: 0.9987 - val_loss: 0.1768
Epoch 4/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9987 - loss: 0.1515 - val_accuracy: 0.9987 - val_loss: 0.0998
Epoch 5/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9975 - loss: 0.0875 - val_accuracy: 0.9987 - val_loss: 0.0622


### making predictions

In [32]:
# Predict probabilities
y_pred_proba = model.predict(x_val).flatten()


40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step


In [33]:
# Find the optimal threshold
thresholds = np.arange(0.0, 1.0, 0.01)
scores = [roc_auc_score(y_val, (y_pred_proba > t).astype(int)) for t in thresholds]
optimal_threshold = thresholds[np.argmax(scores)]
print(f"Optimal threshold: {optimal_threshold}")


Optimal threshold: 0.05


In [35]:

# Apply the optimal threshold
y_pred_optimized = (y_pred_proba > optimal_threshold).astype(int)

In [36]:

# Classification report
print("Optimized Classification Report:")
print(classification_report(y_val, y_pred_optimized, target_names=['Not Fraud', 'Fraud'], zero_division=1))
print('Optimized ROC AUC Score:', roc_auc_score(y_val, y_pred_proba))


Optimized Classification Report:
              precision    recall  f1-score   support

   Not Fraud       1.00      0.98      0.99      1278
       Fraud       0.07      1.00      0.12         2

    accuracy                           0.98      1280
   macro avg       0.53      0.99      0.56      1280
weighted avg       1.00      0.98      0.99      1280

Optimized ROC AUC Score: 0.9917840375586854


# Changing the dataset to be balanced for more analysis

In [39]:
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V22,V23,V24,V25,V26,V27,V28,Amount,Class,Time_hours
0,-1.071719,-4.343904,-0.497727,-0.277591,-2.482346,-0.472073,0.991747,-0.414207,1.694252,-1.744176,...,0.013647,-1.183439,0.536058,0.064148,-0.016945,-0.179133,0.236036,35.112713,0,10.356389
1,1.314915,-0.980378,-0.032665,-2.770975,-1.047365,-0.705180,-0.491240,-0.076395,0.571959,-0.673573,...,-0.481886,-0.009117,0.009622,0.555844,-0.836622,0.099256,0.014826,26.310000,0,13.365278
2,-1.680526,-0.959150,-1.329705,-1.028361,-0.730922,0.253124,3.308312,-0.480062,0.157630,-2.012017,...,-0.204108,0.517400,-0.611013,-0.445912,-0.409744,0.231809,0.253554,35.112713,0,35.431667
3,2.114488,0.169976,-2.051966,0.070326,0.481642,-1.819440,0.886700,-0.673729,0.077960,-0.006214,...,0.642597,-0.058720,0.128632,0.461792,0.234073,-0.082360,-0.075323,12.920000,0,42.215278
4,-5.277692,3.487837,-1.201844,-0.192904,-0.158582,0.949549,0.467901,-0.788434,4.652615,7.407039,...,-1.077669,0.160074,-1.535126,0.873245,-0.322503,1.383375,0.758996,28.900000,0,18.243056


In [40]:
not_frauds = df.query('Class == 0')
frauds = df.query('Class == 1')

not_frauds['Class'].value_counts(),frauds['Class'].value_counts()

(Class
 0    9983
 Name: count, dtype: int64,
 Class
 1    17
 Name: count, dtype: int64)

In [42]:
balanced_df = pd.concat([frauds, not_frauds.sample(len(frauds), random_state=1)])

balanced_df['Class'].value_counts()

Class
1    17
0    17
Name: count, dtype: int64

## Trying to reduce bias and variance, and ensuring that the model is exposed to a variety of data samples during training.

In [43]:
balanced_df = balanced_df.sample(frac=1, random_state=1)

balanced_df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V22,V23,V24,V25,V26,V27,V28,Amount,Class,Time_hours
7526,2.132386,0.705608,-3.530759,0.514779,1.527175,-1.716268,1.132791,-0.574214,0.128904,-1.000805,...,0.703910,-0.245076,0.460049,0.920281,-0.216586,-0.026219,-0.025001,1.000000,1,44.664167
6859,-1.010592,0.781708,1.451502,-1.371163,-0.138884,-1.162590,0.819465,-0.397794,1.144299,0.730169,...,-0.346326,0.056942,0.385472,-0.153704,0.688347,0.158357,-0.150530,8.480000,0,17.583889
1543,-24.590245,14.044567,-26.278701,6.320089,-18.224513,-4.609968,-17.681003,16.213627,-3.794093,-8.372753,...,-1.804874,-1.140605,0.152234,1.715997,-0.220471,1.434951,0.422492,99.990000,1,7.551944
4294,1.780400,-0.549832,0.142401,1.447901,-1.015531,-0.205248,-0.763945,0.128296,1.350898,0.134355,...,0.315447,0.224043,-0.077302,-0.408656,-0.732901,0.064627,-0.009896,76.000000,0,46.527778
1580,1.165314,-1.807710,-1.244951,0.324087,0.022850,1.961193,-0.533722,0.565793,0.899962,-0.015535,...,0.686288,-0.115973,-0.916350,-0.589977,0.401263,-0.049626,-0.001783,35.112713,0,43.055278


## Splitting into x and y

In [44]:
x = balanced_df.drop(columns = 'Class', axis=1)

y = balanced_df['Class']

In [45]:
y.value_counts()

Class
1    17
0    17
Name: count, dtype: int64

### Splitting into train and Test

In [46]:
x_train_b, x_test_b, y_train_b, y_test_b = train_test_split(x,y,train_size=0.8)

## Logistic Regression Model

In [47]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report
import warnings
# Filter out future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Creating a pipeline that scales the data and then applies Logistic Regression
pipeline = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000))

# Define the parameter grid for grid search
param_grid = {
    'logisticregression__C': [0.001, 0.01, 0.1, 1, 10, 30, 50, 80, 100],
    'logisticregression__penalty': ['l2'],
    'logisticregression__class_weight': [ None,'balanced'] 
}

# Create GridSearchCV object
grid_search = GridSearchCV(pipeline, param_grid, cv=10, scoring='roc_auc')

# Fit the grid search to the data
grid_search.fit(x_train, y_train)

# Print the best parameters and the corresponding score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation ROC AUC:", grid_search.best_score_)

# Get the best model from grid search
best_model = grid_search.best_estimator_

# Perform cross-validation with the best estimator
scores = cross_val_score(best_model, x_train, y_train, cv=10, scoring='roc_auc') 
print("Cross-Validation ROC AUC Scores:", scores)
print("Average ROC AUC Score:", scores.mean())


# Predict on the test data
y_pred = best_model.predict(x_test)

# Print the classification report for test metrics
print("Test Metrics Report:\n", classification_report(y_test, y_pred, target_names=['Not Fraud', 'Fraud']))

Best Parameters: {'logisticregression__C': 0.01, 'logisticregression__class_weight': None, 'logisticregression__penalty': 'l2'}
Best Cross-Validation ROC AUC: 0.9925636007827789
Cross-Validation ROC AUC Scores: [1.         1.         1.         1.         1.         1.
 0.92563601 1.         1.         1.        ]
Average ROC AUC Score: 0.9925636007827789
Test Metrics Report:
               precision    recall  f1-score   support

   Not Fraud       1.00      1.00      1.00      1997
       Fraud       1.00      0.67      0.80         3

    accuracy                           1.00      2000
   macro avg       1.00      0.83      0.90      2000
weighted avg       1.00      1.00      1.00      2000



## Random Forest Model

In [48]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# Initializing the RandomForestClassifier
randf_model = RandomForestClassifier(n_jobs=-1)


# Perform cross-validation on the training set with a more suitable scoring metric
scores = cross_val_score(randf_model, x_train, y_train, cv=10, scoring='roc_auc', n_jobs=-1)

# Print the scores for each fold and the average score
print("Cross-Validation ROC AUC Scores:", scores)
print("Average ROC AUC Score:", np.mean(scores))

# Define the parameter grid for Random Forest
param_grid_rf = {
    'n_estimators': [10, 50, 100],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 4],
    'min_samples_leaf': [1, 2]
}

# Using RandomizedSearchCV
random_search_rf = RandomizedSearchCV(RandomForestClassifier(n_jobs=-1), 
                                      param_grid_rf, n_iter=10, cv=5, 
                                      scoring='roc_auc', n_jobs=-1, random_state=42)

random_search_rf.fit(x_train, y_train)

# Print the best parameters and score
print("Best Parameters:", random_search_rf.best_params_)
print("Best ROC AUC:", random_search_rf.best_score_)

# Get the best model from grid search
best_rf_model = random_search_rf.best_estimator_


# Predict on the test data
y_pred = best_rf_model.predict(x_test)



# Print the classification report for test metrics
print("Test Metrics Report:\n", classification_report(y_test, y_pred, 
                                        target_names=['Not Fraud', 'Fraud']))


Cross-Validation ROC AUC Scores: [0.99510763 1.         1.         1.         1.         1.
 0.5        1.         1.         1.        ]
Average ROC AUC Score: 0.9495107632093933
Best Parameters: {'n_estimators': 10, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 10}
Best ROC AUC: 0.949853228962818
Test Metrics Report:
               precision    recall  f1-score   support

   Not Fraud       1.00      1.00      1.00      1997
       Fraud       0.75      1.00      0.86         3

    accuracy                           1.00      2000
   macro avg       0.88      1.00      0.93      2000
weighted avg       1.00      1.00      1.00      2000



## Gradient Boosting Model

In [49]:
from sklearn.ensemble import GradientBoostingClassifier

# Initializing the GradientBoostingClassifier
gbc = GradientBoostingClassifier(n_estimators=50, learning_rate=1.5, max_depth=1, random_state=0)
gbc.fit(x_train, y_train)

# Perform cross-validation on the training set
scores = cross_val_score(gbc, x_train, y_train, cv=5, scoring='roc_auc')  # Using ROC AUC as scoring metric

print("Cross-Validation Scores:", scores)
print("Average ROC AUC Score:", scores.mean())

# Define the parameter grid for grid search
param_grid_gb = {
    'n_estimators': [10,50, 90],
    'learning_rate': [0.1],
    'max_depth': [3, 5],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

# Using RandomizedSearchCV
random_search_gb = RandomizedSearchCV(GradientBoostingClassifier(), param_grid_gb, 
                                      n_iter=10, cv=3, scoring='roc_auc', n_jobs=-1, random_state=42)
random_search_gb.fit(x_train, y_train)

print("Best Parameters:", random_search_gb.best_params_)
print("Best Cross-Validation ROC AUC:", random_search_gb.best_score_)

best_gb_model = random_search_gb.best_estimator_
y_pred_gb = random_search_gb.predict(x_test)

# Assuming you have the predicted probabilities for the positive class
y_pred_proba_gb = best_gb_model.predict_proba(x_test)[:, 1]
print("Test ROC AUC:", roc_auc_score(y_test, y_pred_proba_gb))

print("Test Metrics Report:\n", classification_report(y_test, y_pred_gb, target_names=['Not Fraud', 'Fraud']))

Cross-Validation Scores: [0.74437378 0.99951076 0.99926614 0.5        1.        ]
Average ROC AUC Score: 0.8486301369863014
Best Parameters: {'n_estimators': 50, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': 5, 'learning_rate': 0.1}
Best Cross-Validation ROC AUC: 0.9024108932317727
Test ROC AUC: 0.6664997496244367
Test Metrics Report:
               precision    recall  f1-score   support

   Not Fraud       1.00      1.00      1.00      1997
       Fraud       0.67      0.67      0.67         3

    accuracy                           1.00      2000
   macro avg       0.83      0.83      0.83      2000
weighted avg       1.00      1.00      1.00      2000



## A neural network

In [50]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight

# Splitting the data
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(enumerate(class_weights))

# Building the model
model = Sequential([
    Dense(256, activation='relu', input_shape=(x_train.shape[1],)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), 
              loss='binary_crossentropy', 
              metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])

# Train the model
history = model.fit(x_train, y_train, 
                    epochs=30, 
                    batch_size=32, 
                    validation_data=(x_val, y_val), 
                    class_weight=class_weights_dict, 
                    verbose=1)

# Predict probabilities
y_pred_proba = model.predict(x_val).flatten()

# Find the optimal threshold
thresholds = np.arange(0.0, 1.0, 0.01)
scores = [roc_auc_score(y_val, (y_pred_proba > t).astype(int)) for t in thresholds]
optimal_threshold = thresholds[np.argmax(scores)]
print(f"Optimal threshold: {optimal_threshold}")


# Apply the optimal threshold
y_pred_optimized = (y_pred_proba > optimal_threshold).astype(int)


# Classification report
print("Optimized Classification Report:")
print(classification_report(y_val, y_pred_optimized, target_names=['Not Fraud', 'Fraud'], zero_division=1))
print('Optimized ROC AUC Score:', roc_auc_score(y_val, y_pred_proba))


C:\Users\joshu\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
128/128 ━━━━━━━━━━━━━━━━━━━━ 29s 60ms/step - accuracy: 0.5218 - auc: 0.6297 - loss: 1.1087 - val_accuracy: 0.7002 - val_auc: 1.0000 - val_loss: 0.5855
Epoch 2/30
128/128 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.5767 - auc: 0.6652 - loss: 0.6142 - val_accuracy: 0.7178 - val_auc: 1.0000 - val_loss: 0.5598
Epoch 3/30
128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.6006 - auc: 0.8060 - loss: 0.5446 - val_accuracy: 0.8379 - val_auc: 1.0000 - val_loss: 0.4755
Epoch 4/30
128/128 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.6431 - auc: 0.8090 - loss: 0.4797 - val_accuracy: 0.9590 - val_auc: 1.0000 - val_loss: 0.3972
Epoch 5/30
128/128 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.6952 - auc: 0.7283 - loss: 0.5242 - val_accuracy: 0.9531 - val_auc: 1.0000 - val_loss: 0.3132
Epoch 6/30
128/128 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.7246 - auc: 0.9285 - loss: 0.4022 - val_accuracy: 0.9463 - val_auc: 1.0000 - val_loss: 0.2993
Epoch 7/30
128/128 ━━━━━━━━━━━━━━

# Conclusion:

The Neural Network model demonstrates the best overall performance, particularly in handling the balanced dataset, where it achieves perfect precision and recall for the 'Fraud' class and an ROC AUC score of 1.0. This indicates an exceptional ability to distinguish between fraudulent and non-fraudulent transactions without any false positives or false negatives.